In [ ]:

# %%
# Install pygame if missing (Jupyter magic)
# force reinstall a known-good binary and invalidate import caches to avoid pygame.base issues
%pip install --upgrade --force-reinstall --no-cache-dir pygame==2.1.3

import importlib
importlib.invalidate_caches()

import sys
import random
import pygame
from pygame import sprite
from pygame.math import Vector2

# --- constants ---
CELL_SIZE = 24
COLUMNS = 10
ROWS = 20
PADDING = 10
LIGHT_COLOR = (200, 200, 200)
GRAY = (30, 30, 30)
GAME_WIDTH = COLUMNS * CELL_SIZE
GAME_HEIGHT = ROWS * CELL_SIZE
WINDOW_WIDTH = GAME_WIDTH + PADDING * 2
WINDOW_HEIGHT = GAME_HEIGHT + PADDING * 2
UPDATE_START_SPEED = 1000  # milliseconds
MOVE_WAIT_TIME = 150  # milliseconds
ROTATE_WAIT_TIME = 150

# --- small Timer helper ---
class Timer:
    def __init__(self, interval, repeat, callback):
        self.interval = int(interval)
        self.repeat = bool(repeat)
        self.callback = callback
        self.active = False
        self._last = pygame.time.get_ticks()

    def activate(self):
        self.active = True
        self._last = pygame.time.get_ticks()

    def update(self):
        if not self.active:
            return
        now = pygame.time.get_ticks()
        if now - self._last >= self.interval:
            self._last = now
            if callable(self.callback):
                self.callback()
            if not self.repeat:
                self.active = False

# --- Block sprite ---
class Block(sprite.Sprite):
    def __init__(self, group, pos, color):
        super().__init__(group)
        self.image = pygame.Surface((CELL_SIZE, CELL_SIZE))
        self.image.fill(color)
        # pos is (x, y) in grid coordinates
        self.pos = Vector2(pos)
        self.rect = self.image.get_rect(topleft=(int(self.pos.x * CELL_SIZE), int(self.pos.y * CELL_SIZE)))

    def rotate_around(self, pivot_pos):
        # rotate 90 degrees clockwise around pivot
        rel = self.pos - pivot_pos
        rotated = rel.rotate(-90)  # pygame Vector2 rotate is counter-clockwise with positive angle
        return pivot_pos + rotated

    def horizontal_collide(self, x, field_data):
        # check bounds and occupancy
        if not (0 <= x < COLUMNS):
            return True
        y = int(self.pos.y)
        if 0 <= y < ROWS and field_data[y][x]:
            return True
        return False

    def update(self):
        self.rect.topleft = (int(self.pos.x * CELL_SIZE), int(self.pos.y * CELL_SIZE))

# --- tetromino definitions ---
TETROMINOS = {
    # shapes are lists of Vector2 offsets relative to origin
    'L': {'shape': [Vector2(0, 0), Vector2(0, 1), Vector2(0, 2), Vector2(1, 2)], 'color': (255, 165, 0)},
    'O': {'shape': [Vector2(0, 0), Vector2(1, 0), Vector2(0, 1), Vector2(1, 1)], 'color': (255, 215, 0)},
    'I': {'shape': [Vector2(0, 0), Vector2(0, 1), Vector2(0, 2), Vector2(0, 3)], 'color': (0, 240, 240)},
    'T': {'shape': [Vector2(1, 0), Vector2(0, 1), Vector2(1, 1), Vector2(2, 1)], 'color': (160, 32, 240)},
}

class Tetromino:
    def __init__(self, shape_key, group, create_new_tetromino, field_data, origin=(3, 0)):
        self.group = group
        self.shape_key = shape_key
        template = TETROMINOS[self.shape_key]
        self.shape = template['shape']
        self.color = template['color']
        self.create_new_tetromino = create_new_tetromino
        self.field_data = field_data
        self.origin = Vector2(origin)
        # create blocks from template positions
        self.blocks = []
        for p in self.shape:
            pos = Vector2(self.origin.x + p.x, self.origin.y + p.y)
            b = Block(self.group, (pos.x, pos.y), self.color)
            self.blocks.append(b)

    def next_move_horizontal(self, amount):
        for b in self.blocks:
            x = int(b.pos.x + amount)
            if b.horizontal_collide(x, self.field_data):
                return True
        return False

    def move_horizontal(self, amount):
        if not self.next_move_horizontal(amount):
            for b in self.blocks:
                b.pos.x += amount

    def next_move_vertical(self, amount):
        for b in self.blocks:
            new_y = int(b.pos.y + amount)
            x = int(b.pos.x)
            if new_y >= ROWS:
                return True
            if 0 <= new_y < ROWS and 0 <= x < COLUMNS and self.field_data[new_y][x]:
                return True
        return False

    def move_down(self):
        if not self.next_move_vertical(1):
            for b in self.blocks:
                b.pos.y += 1
            return True
        else:
            # lock into field
            for block in self.blocks:
                y = int(block.pos.y)
                x = int(block.pos.x)
                if 0 <= y < ROWS and 0 <= x < COLUMNS:
                    self.field_data[y][x] = 1
            # remove blocks from sprite group (they are now "locked" and could be replaced with permanent blocks if desired)
            for b in self.blocks:
                b.kill()
            self.create_new_tetromino()
            return False

    def rotate(self):
        # 'O' doesn't rotate
        if self.shape_key == 'O':
            return False
        pivot = self.blocks[0].pos
        new_positions = [b.rotate_around(pivot) for b in self.blocks]
        # collision check
        for pos in new_positions:
            x, y = int(pos.x), int(pos.y)
            if not (0 <= x < COLUMNS and 0 <= y < ROWS):
                return False
            if self.field_data[y][x]:
                return False
        # apply
        for b, pos in zip(self.blocks, new_positions):
            b.pos = pos
        return True

# --- Game class ---
class Game:
    def __init__(self):
        pygame.init()
        self.display_surface = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
        pygame.display.set_caption("Tetris (minimal)")
        self.surface = pygame.Surface((GAME_WIDTH, GAME_HEIGHT))
        self.lines_surface = pygame.Surface(self.surface.get_size(), flags=pygame.SRCALPHA)
        self.sprites = sprite.Group()
        self.field_data = [[0 for _ in range(COLUMNS)] for _ in range(ROWS)]
        self.timers = {}
        self.down_speed = UPDATE_START_SPEED
        self.down_speed_faster = int(self.down_speed * 0.3)
        self.down_pressed = False
        self.clock = pygame.time.Clock()
        self.tetromino = None
        self.create_new_tetromino()

        self.timers = {
            'vertical_move': Timer(self.down_speed, True, self.auto_move_down),
            'horizontal_move': Timer(MOVE_WAIT_TIME, True, lambda: None),
            'rotate': Timer(ROTATE_WAIT_TIME, True, lambda: None)
        }
        self.timers['vertical_move'].activate()

    def create_new_tetromino(self):
        # spawn a new tetromino near top
        shape = random.choice(list(TETROMINOS.keys()))
        self.tetromino = Tetromino(shape, self.sprites, self.create_new_tetromino, self.field_data, origin=(3, 0))

    def auto_move_down(self):
        # called by vertical timer
        if self.tetromino:
            self.tetromino.move_down()

    def draw_border(self):
        border_rect = pygame.Rect(PADDING, PADDING, self.surface.get_width(), self.surface.get_height())
        pygame.draw.rect(self.display_surface, LIGHT_COLOR, border_rect, 1)

    def clear_lines_surface(self):
        self.lines_surface.fill((0, 0, 0, 0))

    def draw_grid(self):
        self.clear_lines_surface()
        for col in range(COLUMNS):
            x = col * CELL_SIZE
            pygame.draw.line(self.lines_surface, LIGHT_COLOR, (x, 0), (x, self.surface.get_height()), 1)
        for row in range(ROWS):
            y = row * CELL_SIZE
            pygame.draw.line(self.lines_surface, LIGHT_COLOR, (0, y), (self.surface.get_width(), y), 1)
        self.surface.blit(self.lines_surface, (0, 0))

    def input(self):
        keys = pygame.key.get_pressed()
        # horizontal
        if not self.timers['horizontal_move'].active:
            if keys[pygame.K_LEFT]:
                if self.tetromino:
                    self.tetromino.move_horizontal(-1)
                self.timers['horizontal_move'].activate()
            if keys[pygame.K_RIGHT]:
                if self.tetromino:
                    self.tetromino.move_horizontal(1)
                self.timers['horizontal_move'].activate()

        # rotate
        if not self.timers['rotate'].active:
            if keys[pygame.K_UP]:
                if self.tetromino:
                    self.tetromino.rotate()
                self.timers['rotate'].activate()

        # soft drop
        if keys[pygame.K_DOWN]:
            if not self.down_pressed:
                self.timers['vertical_move'].interval = self.down_speed_faster
                self.down_pressed = True
        else:
            if self.down_pressed:
                self.timers['vertical_move'].interval = self.down_speed
                self.down_pressed = False

    def check_finished_rows(self):
        delete_rows = [i for i, row in enumerate(self.field_data) if all(cell == 1 for cell in row)]
        if not delete_rows:
            return
        # remove rows and shift above down
        for dr in sorted(delete_rows):
            del self.field_data[dr]
            self.field_data.insert(0, [0 for _ in range(COLUMNS)])
        # (optional) could update sprites/locked blocks here

    def update(self, dt: float = 0):
        for timer in self.timers.values():
            timer.update()
        self.sprites.update()
        self.check_finished_rows()

    def run(self):
        running = True
        while running:
            dt = self.clock.tick(60) / 1000.0
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
            self.input()
            self.update(dt)
            self.surface.fill(GRAY)
            self.sprites.draw(self.surface)
            self.draw_grid()
            self.draw_border()
            self.display_surface.fill((0, 0, 0))
            self.display_surface.blit(self.surface, (PADDING, PADDING))
            pygame.display.flip()
        pygame.quit()

# run interactively when cell is executed
if __name__ == '__main__':
    Game().run()




ModuleNotFoundError: No module named 'pygame.base'